In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import networkx as nx

import rpy2 as rp
import rpy2.robjects as ro
from rpy2.robjects import pandas2ri
import pyreadr

import os
import time
import h5py
# import mmdet
import scanpy
# import ssam
import pickle
import random
# import pyarrow
import dgl
import ast

from tqdm import tqdm

from datetime import datetime

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, recall_score, precision_score, f1_score, accuracy_score, matthews_corrcoef, mean_squared_error

# from captum.attr import IntegratedGradients
# from captum.attr import LayerConductance
# from captum.attr import NeuronConductance

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torch.nn.functional as F

# import torch_geometric

from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torchvision import transforms, utils

from skimage import io
from skimage import img_as_float
from scipy import ndimage

# from ssam.utils import corr, calc_ctmap, calc_corrmap, flood_fill, calc_kde
from scipy.spatial.distance import cdist

from collections import OrderedDict

from packaging import version
import subprocess

<h1>Take Matrix

In [2]:
# read from rds
mat_151507 = pyreadr.read_r('dataset/df_logcount_507.rds')
mat_151672 = pyreadr.read_r('dataset/df_logcount_672.rds')
mat_151673 = pyreadr.read_r('dataset/df_logcount_673.rds')
spat_507 = np.load('dataset/datapoints/com3_encoded_507.npy') 
spat_672 = np.load('dataset/datapoints/com3_encoded_672.npy') 
spat_673 = np.load('dataset/datapoints/com3_encoded_673.npy') 

In [3]:
spat_507.shape

(6101, 1, 4226)

In [4]:
# dict to dataframe
df_151507 = mat_151507[None]
df_151672 = mat_151672[None]
df_151673 = mat_151673[None]
combined_507_672 = pd.concat([df_151507,df_151672],axis=1)
combined_all = pd.concat([combined_507_672,df_151673],axis=1)

In [5]:
df_spat507 = pd.DataFrame(spat_507.reshape((spat_507.shape[0],spat_507.shape[-1])))
df_spat672 = pd.DataFrame(spat_672.reshape((spat_672.shape[0],spat_672.shape[-1])))
df_spat673 = pd.DataFrame(spat_673.reshape((spat_673.shape[0],spat_673.shape[-1])))

In [6]:
df_spat507.shape

(6101, 4226)

In [8]:
df_spat507

,0,1,2,3,4,5,6,7,8,9,...,4216,4217,4218,4219,4220,4221,4222,4223,4224,4225
0,0.497303,0.503828,0.499100,0.506157,0.503188,0.501808,0.501589,0.501911,0.503325,0.501535,...,0.501724,0.499947,0.501404,0.500734,0.500185,0.503060,0.501716,0.497402,0.506611,0.500518
1,0.497519,0.506479,0.501702,0.508399,0.503553,0.503084,0.503082,0.504586,0.504508,0.501807,...,0.504006,0.502240,0.503011,0.503235,0.503441,0.503557,0.503656,0.499215,0.510807,0.502156
2,0.497514,0.504594,0.499708,0.506777,0.503338,0.502211,0.502212,0.502457,0.503486,0.501545,...,0.502166,0.500418,0.502160,0.501544,0.500958,0.503299,0.502055,0.497911,0.507778,0.500941
3,0.497514,0.506050,0.501324,0.508056,0.503539,0.502858,0.502835,0.504175,0.504373,0.501797,...,0.503596,0.501905,0.502692,0.502827,0.502997,0.503392,0.503385,0.498933,0.510172,0.501887
4,0.497361,0.504044,0.499176,0.506373,0.503248,0.501896,0.501689,0.502047,0.503510,0.501644,...,0.501967,0.500119,0.501492,0.500806,0.500351,0.503002,0.501845,0.497385,0.506906,0.500702
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6096,0.497334,0.503891,0.499153,0.506201,0.503188,0.501855,0.501657,0.501950,0.503317,0.501513,...,0.501756,0.499976,0.501489,0.500847,0.500247,0.503105,0.501736,0.497467,0.506722,0.500552
6097,0.497661,0.507707,0.502830,0.509500,0.503657,0.503860,0.503999,0.505746,0.505054,0.502011,...,0.505105,0.503282,0.503941,0.504523,0.504824,0.503993,0.504454,0.500015,0.512780,0.502956
6098,0.497686,0.507720,0.502860,0.509536,0.503714,0.503866,0.504025,0.505762,0.505093,0.502010,...,0.505115,0.503312,0.503957,0.504516,0.504867,0.504010,0.504457,0.500040,0.512802,0.502974
6099,0.497463,0.504431,0.499572,0.506608,0.503313,0.502132,0.502016,0.502314,0.503444,0.501497,...,0.502044,0.500317,0.501968,0.501336,0.500796,0.503193,0.501944,0.497809,0.507530,0.500799


In [7]:
tots = df_151507.shape[1] + df_151672.shape[1] + df_151673.shape[1]
tots

11880

In [9]:
df_spat507.columns = df_151507.columns
df_spat672.columns = df_151672.columns
df_spat673.columns = df_151673.columns

In [10]:
df_spat507

,AAACAACGAATAGTTC-1,AAACAAGTATCTCCCA-1,AAACAATCTACTAGCA-1,AAACACCAATAACTGC-1,AAACAGCTTTCAGAAG-1,AAACAGGGTCTATATT-1,AAACAGTGTTCCTGGG-1,AAACATTTCCCGGATT-1,AAACCACTACACAGAT-1,AAACCCGAACGAAATC-1,...,TTGTGTATGCCACCAA-1,TTGTGTTTCCCGAAAG-1,TTGTTAGCAAATTCGA-1,TTGTTCAGTGTGCTAC-1,TTGTTCTAGATACGCT-1,TTGTTGTGTGTCAAGA-1,TTGTTTCACATCCAGG-1,TTGTTTCATTAGTCTA-1,TTGTTTCCATACAACT-1,TTGTTTGTGTAAATTC-1
0,0.497303,0.503828,0.499100,0.506157,0.503188,0.501808,0.501589,0.501911,0.503325,0.501535,...,0.501724,0.499947,0.501404,0.500734,0.500185,0.503060,0.501716,0.497402,0.506611,0.500518
1,0.497519,0.506479,0.501702,0.508399,0.503553,0.503084,0.503082,0.504586,0.504508,0.501807,...,0.504006,0.502240,0.503011,0.503235,0.503441,0.503557,0.503656,0.499215,0.510807,0.502156
2,0.497514,0.504594,0.499708,0.506777,0.503338,0.502211,0.502212,0.502457,0.503486,0.501545,...,0.502166,0.500418,0.502160,0.501544,0.500958,0.503299,0.502055,0.497911,0.507778,0.500941
3,0.497514,0.506050,0.501324,0.508056,0.503539,0.502858,0.502835,0.504175,0.504373,0.501797,...,0.503596,0.501905,0.502692,0.502827,0.502997,0.503392,0.503385,0.498933,0.510172,0.501887
4,0.497361,0.504044,0.499176,0.506373,0.503248,0.501896,0.501689,0.502047,0.503510,0.501644,...,0.501967,0.500119,0.501492,0.500806,0.500351,0.503002,0.501845,0.497385,0.506906,0.500702
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6096,0.497334,0.503891,0.499153,0.506201,0.503188,0.501855,0.501657,0.501950,0.503317,0.501513,...,0.501756,0.499976,0.501489,0.500847,0.500247,0.503105,0.501736,0.497467,0.506722,0.500552
6097,0.497661,0.507707,0.502830,0.509500,0.503657,0.503860,0.503999,0.505746,0.505054,0.502011,...,0.505105,0.503282,0.503941,0.504523,0.504824,0.503993,0.504454,0.500015,0.512780,0.502956
6098,0.497686,0.507720,0.502860,0.509536,0.503714,0.503866,0.504025,0.505762,0.505093,0.502010,...,0.505115,0.503312,0.503957,0.504516,0.504867,0.504010,0.504457,0.500040,0.512802,0.502974
6099,0.497463,0.504431,0.499572,0.506608,0.503313,0.502132,0.502016,0.502314,0.503444,0.501497,...,0.502044,0.500317,0.501968,0.501336,0.500796,0.503193,0.501944,0.497809,0.507530,0.500799


In [11]:
df_spat672

,AAACAAGTATCTCCCA-1,AAACACCAATAACTGC-1,AAACAGAGCGACTCCT-1,AAACAGCTTTCAGAAG-1,AAACAGGGTCTATATT-1,AAACATTTCCCGGATT-1,AAACCACTACACAGAT-1,AAACCCGAACGAAATC-1,AAACCGGAAATGTTAA-1,AAACCGGGTAGGTACC-1,...,TTGTGTATGCCACCAA-1,TTGTGTTTCCCGAAAG-1,TTGTTAGCAAATTCGA-1,TTGTTCAGTGTGCTAC-1,TTGTTCTAGATACGCT-1,TTGTTGTGTGTCAAGA-1,TTGTTTCACATCCAGG-1,TTGTTTCATTAGTCTA-1,TTGTTTCCATACAACT-1,TTGTTTGTGTAAATTC-1
0,0.500941,0.501431,0.500735,0.502163,0.500962,0.502752,0.501171,0.503403,0.504747,0.501785,...,0.496879,0.499353,0.498534,0.497914,0.502389,0.501344,0.501564,0.499603,0.500980,0.502145
1,0.502217,0.501440,0.501322,0.502943,0.501841,0.503173,0.501717,0.502139,0.506421,0.501451,...,0.496428,0.499017,0.499361,0.499168,0.502300,0.501193,0.503044,0.500152,0.500968,0.502331
2,0.500900,0.501432,0.500702,0.502129,0.500907,0.502720,0.501172,0.503435,0.504686,0.501815,...,0.496892,0.499419,0.498483,0.497861,0.502442,0.501360,0.501507,0.499627,0.501001,0.502178
3,0.505723,0.502926,0.504121,0.505831,0.505278,0.504169,0.505249,0.499500,0.510995,0.500968,...,0.496205,0.498825,0.503641,0.502374,0.503310,0.501013,0.507015,0.502291,0.502068,0.504918
4,0.501522,0.501275,0.500962,0.502461,0.501276,0.502805,0.501216,0.502801,0.505530,0.501577,...,0.496650,0.499155,0.498737,0.498503,0.502211,0.501240,0.502198,0.499861,0.501022,0.502003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7028,0.501739,0.501291,0.501107,0.502641,0.501463,0.502998,0.501344,0.502501,0.505877,0.501572,...,0.496523,0.499110,0.498933,0.498777,0.502166,0.501147,0.502524,0.499929,0.500936,0.502087
7029,0.500970,0.501421,0.500731,0.502183,0.500989,0.502754,0.501171,0.503376,0.504764,0.501777,...,0.496870,0.499345,0.498544,0.497944,0.502366,0.501345,0.501579,0.499610,0.500983,0.502132
7030,0.500930,0.501422,0.500735,0.502165,0.500949,0.502742,0.501193,0.503398,0.504728,0.501786,...,0.496887,0.499364,0.498520,0.497917,0.502400,0.501349,0.501551,0.499616,0.500991,0.502165
7031,0.502801,0.501807,0.501689,0.503461,0.502465,0.503223,0.502253,0.501839,0.507091,0.501374,...,0.496416,0.498902,0.499943,0.499640,0.502491,0.501320,0.503574,0.500481,0.501086,0.502632


In [15]:
df_spat673

,AAACAAGTATCTCCCA-1,AAACAATCTACTAGCA-1,AAACACCAATAACTGC-1,AAACAGAGCGACTCCT-1,AAACAGCTTTCAGAAG-1,AAACAGGGTCTATATT-1,AAACAGTGTTCCTGGG-1,AAACATTTCCCGGATT-1,AAACCCGAACGAAATC-1,AAACCGGGTAGGTACC-1,...,TTGTGTATGCCACCAA-1,TTGTGTTTCCCGAAAG-1,TTGTTAGCAAATTCGA-1,TTGTTCAGTGTGCTAC-1,TTGTTGTGTGTCAAGA-1,TTGTTTCACATCCAGG-1,TTGTTTCATTAGTCTA-1,TTGTTTCCATACAACT-1,TTGTTTGTATTACACG-1,TTGTTTGTGTAAATTC-1
0,0.497470,0.489888,0.487792,0.499749,0.489667,0.500780,0.502566,0.497517,0.488813,0.489952,...,0.490342,0.494190,0.492384,0.492397,0.494748,0.493126,0.484210,0.489002,0.495627,0.489994
1,0.497681,0.492889,0.490805,0.500105,0.492776,0.500844,0.502220,0.498774,0.491595,0.493936,...,0.493638,0.497006,0.494774,0.494201,0.497792,0.495712,0.488632,0.491977,0.497911,0.492390
2,0.498039,0.495144,0.493367,0.500593,0.495080,0.501343,0.502262,0.499620,0.493385,0.496791,...,0.495884,0.499398,0.496881,0.495691,0.500770,0.497651,0.492032,0.494450,0.499681,0.494847
3,0.497626,0.490970,0.488938,0.499670,0.490886,0.500792,0.502499,0.498163,0.489740,0.491563,...,0.491765,0.495324,0.493268,0.493159,0.495985,0.494170,0.486015,0.490283,0.496488,0.491099
4,0.497793,0.493611,0.491658,0.500050,0.493761,0.501002,0.502409,0.499213,0.492196,0.495005,...,0.494365,0.498114,0.495514,0.494638,0.498925,0.496375,0.489954,0.492890,0.498397,0.493476
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8099,0.497495,0.492010,0.490008,0.499682,0.491960,0.500851,0.502511,0.498564,0.490657,0.492560,...,0.492611,0.496259,0.494127,0.493547,0.497013,0.495098,0.487406,0.491095,0.497117,0.491866
8100,0.497425,0.486293,0.483680,0.499034,0.486513,0.500494,0.503179,0.496796,0.485055,0.485529,...,0.486323,0.491544,0.489249,0.490813,0.490854,0.490435,0.479145,0.485391,0.492681,0.487194
8101,0.497606,0.489347,0.487179,0.499672,0.489253,0.500369,0.502435,0.497361,0.487996,0.489399,...,0.489216,0.493626,0.492013,0.492649,0.494078,0.493263,0.483633,0.488475,0.495655,0.489331
8102,0.497744,0.493454,0.491393,0.500146,0.493497,0.500968,0.502462,0.499110,0.491980,0.494518,...,0.494085,0.497719,0.495215,0.494561,0.498651,0.496119,0.489558,0.492707,0.498208,0.493146


In [12]:
df_151507

,AAACAACGAATAGTTC-1,AAACAAGTATCTCCCA-1,AAACAATCTACTAGCA-1,AAACACCAATAACTGC-1,AAACAGCTTTCAGAAG-1,AAACAGGGTCTATATT-1,AAACAGTGTTCCTGGG-1,AAACATTTCCCGGATT-1,AAACCACTACACAGAT-1,AAACCCGAACGAAATC-1,...,TTGTGTATGCCACCAA-1,TTGTGTTTCCCGAAAG-1,TTGTTAGCAAATTCGA-1,TTGTTCAGTGTGCTAC-1,TTGTTCTAGATACGCT-1,TTGTTGTGTGTCAAGA-1,TTGTTTCACATCCAGG-1,TTGTTTCATTAGTCTA-1,TTGTTTCCATACAACT-1,TTGTTTGTGTAAATTC-1
rownames,,,,,,,,,,,,,,,,,,,,,
ENSG00000243485,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000237613,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000186092,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000238009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000239945,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000277856,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000275063,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000271254,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
df_151672

,AAACAAGTATCTCCCA-1,AAACACCAATAACTGC-1,AAACAGAGCGACTCCT-1,AAACAGCTTTCAGAAG-1,AAACAGGGTCTATATT-1,AAACATTTCCCGGATT-1,AAACCACTACACAGAT-1,AAACCCGAACGAAATC-1,AAACCGGAAATGTTAA-1,AAACCGGGTAGGTACC-1,...,TTGTGTATGCCACCAA-1,TTGTGTTTCCCGAAAG-1,TTGTTAGCAAATTCGA-1,TTGTTCAGTGTGCTAC-1,TTGTTCTAGATACGCT-1,TTGTTGTGTGTCAAGA-1,TTGTTTCACATCCAGG-1,TTGTTTCATTAGTCTA-1,TTGTTTCCATACAACT-1,TTGTTTGTGTAAATTC-1
rownames,,,,,,,,,,,,,,,,,,,,,
ENSG00000243485,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000237613,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000186092,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000238009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000239945,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000277856,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000275063,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000271254,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
df_151673

,AAACAAGTATCTCCCA-1,AAACAATCTACTAGCA-1,AAACACCAATAACTGC-1,AAACAGAGCGACTCCT-1,AAACAGCTTTCAGAAG-1,AAACAGGGTCTATATT-1,AAACAGTGTTCCTGGG-1,AAACATTTCCCGGATT-1,AAACCCGAACGAAATC-1,AAACCGGGTAGGTACC-1,...,TTGTGTATGCCACCAA-1,TTGTGTTTCCCGAAAG-1,TTGTTAGCAAATTCGA-1,TTGTTCAGTGTGCTAC-1,TTGTTGTGTGTCAAGA-1,TTGTTTCACATCCAGG-1,TTGTTTCATTAGTCTA-1,TTGTTTCCATACAACT-1,TTGTTTGTATTACACG-1,TTGTTTGTGTAAATTC-1
rownames,,,,,,,,,,,,,,,,,,,,,
ENSG00000243485,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000237613,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000186092,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000238009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000239945,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000277856,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000275063,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000271254,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
embed_507 = pd.concat([df_151507,df_spat507], axis=0)
embed_507

,AAACAACGAATAGTTC-1,AAACAAGTATCTCCCA-1,AAACAATCTACTAGCA-1,AAACACCAATAACTGC-1,AAACAGCTTTCAGAAG-1,AAACAGGGTCTATATT-1,AAACAGTGTTCCTGGG-1,AAACATTTCCCGGATT-1,AAACCACTACACAGAT-1,AAACCCGAACGAAATC-1,...,TTGTGTATGCCACCAA-1,TTGTGTTTCCCGAAAG-1,TTGTTAGCAAATTCGA-1,TTGTTCAGTGTGCTAC-1,TTGTTCTAGATACGCT-1,TTGTTGTGTGTCAAGA-1,TTGTTTCACATCCAGG-1,TTGTTTCATTAGTCTA-1,TTGTTTCCATACAACT-1,TTGTTTGTGTAAATTC-1
ENSG00000243485,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000237613,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000186092,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000238009,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000239945,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6096,0.497334,0.503891,0.499153,0.506201,0.503188,0.501855,0.501657,0.501950,0.503317,0.501513,...,0.501756,0.499976,0.501489,0.500847,0.500247,0.503105,0.501736,0.497467,0.506722,0.500552
6097,0.497661,0.507707,0.502830,0.509500,0.503657,0.503860,0.503999,0.505746,0.505054,0.502011,...,0.505105,0.503282,0.503941,0.504523,0.504824,0.503993,0.504454,0.500015,0.512780,0.502956
6098,0.497686,0.507720,0.502860,0.509536,0.503714,0.503866,0.504025,0.505762,0.505093,0.502010,...,0.505115,0.503312,0.503957,0.504516,0.504867,0.504010,0.504457,0.500040,0.512802,0.502974
6099,0.497463,0.504431,0.499572,0.506608,0.503313,0.502132,0.502016,0.502314,0.503444,0.501497,...,0.502044,0.500317,0.501968,0.501336,0.500796,0.503193,0.501944,0.497809,0.507530,0.500799


In [17]:
embed_672 = pd.concat([df_151672,df_spat672], axis=0)
embed_672

,AAACAAGTATCTCCCA-1,AAACACCAATAACTGC-1,AAACAGAGCGACTCCT-1,AAACAGCTTTCAGAAG-1,AAACAGGGTCTATATT-1,AAACATTTCCCGGATT-1,AAACCACTACACAGAT-1,AAACCCGAACGAAATC-1,AAACCGGAAATGTTAA-1,AAACCGGGTAGGTACC-1,...,TTGTGTATGCCACCAA-1,TTGTGTTTCCCGAAAG-1,TTGTTAGCAAATTCGA-1,TTGTTCAGTGTGCTAC-1,TTGTTCTAGATACGCT-1,TTGTTGTGTGTCAAGA-1,TTGTTTCACATCCAGG-1,TTGTTTCATTAGTCTA-1,TTGTTTCCATACAACT-1,TTGTTTGTGTAAATTC-1
ENSG00000243485,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000237613,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000186092,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000238009,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000239945,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7028,0.501739,0.501291,0.501107,0.502641,0.501463,0.502998,0.501344,0.502501,0.505877,0.501572,...,0.496523,0.499110,0.498933,0.498777,0.502166,0.501147,0.502524,0.499929,0.500936,0.502087
7029,0.500970,0.501421,0.500731,0.502183,0.500989,0.502754,0.501171,0.503376,0.504764,0.501777,...,0.496870,0.499345,0.498544,0.497944,0.502366,0.501345,0.501579,0.499610,0.500983,0.502132
7030,0.500930,0.501422,0.500735,0.502165,0.500949,0.502742,0.501193,0.503398,0.504728,0.501786,...,0.496887,0.499364,0.498520,0.497917,0.502400,0.501349,0.501551,0.499616,0.500991,0.502165
7031,0.502801,0.501807,0.501689,0.503461,0.502465,0.503223,0.502253,0.501839,0.507091,0.501374,...,0.496416,0.498902,0.499943,0.499640,0.502491,0.501320,0.503574,0.500481,0.501086,0.502632


In [18]:
embed_673 = pd.concat([df_151673,df_spat673], axis=0)
embed_673

,AAACAAGTATCTCCCA-1,AAACAATCTACTAGCA-1,AAACACCAATAACTGC-1,AAACAGAGCGACTCCT-1,AAACAGCTTTCAGAAG-1,AAACAGGGTCTATATT-1,AAACAGTGTTCCTGGG-1,AAACATTTCCCGGATT-1,AAACCCGAACGAAATC-1,AAACCGGGTAGGTACC-1,...,TTGTGTATGCCACCAA-1,TTGTGTTTCCCGAAAG-1,TTGTTAGCAAATTCGA-1,TTGTTCAGTGTGCTAC-1,TTGTTGTGTGTCAAGA-1,TTGTTTCACATCCAGG-1,TTGTTTCATTAGTCTA-1,TTGTTTCCATACAACT-1,TTGTTTGTATTACACG-1,TTGTTTGTGTAAATTC-1
ENSG00000243485,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000237613,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000186092,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000238009,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000239945,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8099,0.497495,0.492010,0.490008,0.499682,0.491960,0.500851,0.502511,0.498564,0.490657,0.492560,...,0.492611,0.496259,0.494127,0.493547,0.497013,0.495098,0.487406,0.491095,0.497117,0.491866
8100,0.497425,0.486293,0.483680,0.499034,0.486513,0.500494,0.503179,0.496796,0.485055,0.485529,...,0.486323,0.491544,0.489249,0.490813,0.490854,0.490435,0.479145,0.485391,0.492681,0.487194
8101,0.497606,0.489347,0.487179,0.499672,0.489253,0.500369,0.502435,0.497361,0.487996,0.489399,...,0.489216,0.493626,0.492013,0.492649,0.494078,0.493263,0.483633,0.488475,0.495655,0.489331
8102,0.497744,0.493454,0.491393,0.500146,0.493497,0.500968,0.502462,0.499110,0.491980,0.494518,...,0.494085,0.497719,0.495215,0.494561,0.498651,0.496119,0.489558,0.492707,0.498208,0.493146


In [19]:
embed_507_672 = pd.concat([embed_507,embed_672],axis=1)
embed_507_672

,AAACAACGAATAGTTC-1,AAACAAGTATCTCCCA-1,AAACAATCTACTAGCA-1,AAACACCAATAACTGC-1,AAACAGCTTTCAGAAG-1,AAACAGGGTCTATATT-1,AAACAGTGTTCCTGGG-1,AAACATTTCCCGGATT-1,AAACCACTACACAGAT-1,AAACCCGAACGAAATC-1,...,TTGTGTATGCCACCAA-1,TTGTGTTTCCCGAAAG-1,TTGTTAGCAAATTCGA-1,TTGTTCAGTGTGCTAC-1,TTGTTCTAGATACGCT-1,TTGTTGTGTGTCAAGA-1,TTGTTTCACATCCAGG-1,TTGTTTCATTAGTCTA-1,TTGTTTCCATACAACT-1,TTGTTTGTGTAAATTC-1
ENSG00000243485,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000237613,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000186092,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000238009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000239945,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7028,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.496523,0.499110,0.498933,0.498777,0.502166,0.501147,0.502524,0.499929,0.500936,0.502087
7029,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.496870,0.499345,0.498544,0.497944,0.502366,0.501345,0.501579,0.499610,0.500983,0.502132
7030,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.496887,0.499364,0.498520,0.497917,0.502400,0.501349,0.501551,0.499616,0.500991,0.502165
7031,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.496416,0.498902,0.499943,0.499640,0.502491,0.501320,0.503574,0.500481,0.501086,0.502632


In [20]:
embed_all = pd.concat([embed_507_672,embed_673],axis=1)
embed_all

,AAACAACGAATAGTTC-1,AAACAAGTATCTCCCA-1,AAACAATCTACTAGCA-1,AAACACCAATAACTGC-1,AAACAGCTTTCAGAAG-1,AAACAGGGTCTATATT-1,AAACAGTGTTCCTGGG-1,AAACATTTCCCGGATT-1,AAACCACTACACAGAT-1,AAACCCGAACGAAATC-1,...,TTGTGTATGCCACCAA-1,TTGTGTTTCCCGAAAG-1,TTGTTAGCAAATTCGA-1,TTGTTCAGTGTGCTAC-1,TTGTTGTGTGTCAAGA-1,TTGTTTCACATCCAGG-1,TTGTTTCATTAGTCTA-1,TTGTTTCCATACAACT-1,TTGTTTGTATTACACG-1,TTGTTTGTGTAAATTC-1
ENSG00000243485,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000237613,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000186092,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000238009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000239945,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8099,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.492611,0.496259,0.494127,0.493547,0.497013,0.495098,0.487406,0.491095,0.497117,0.491866
8100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.486323,0.491544,0.489249,0.490813,0.490854,0.490435,0.479145,0.485391,0.492681,0.487194
8101,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.489216,0.493626,0.492013,0.492649,0.494078,0.493263,0.483633,0.488475,0.495655,0.489331
8102,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.494085,0.497719,0.495215,0.494561,0.498651,0.496119,0.489558,0.492707,0.498208,0.493146


In [21]:
embed_all.fillna(0,inplace=True)
embed_all

,AAACAACGAATAGTTC-1,AAACAAGTATCTCCCA-1,AAACAATCTACTAGCA-1,AAACACCAATAACTGC-1,AAACAGCTTTCAGAAG-1,AAACAGGGTCTATATT-1,AAACAGTGTTCCTGGG-1,AAACATTTCCCGGATT-1,AAACCACTACACAGAT-1,AAACCCGAACGAAATC-1,...,TTGTGTATGCCACCAA-1,TTGTGTTTCCCGAAAG-1,TTGTTAGCAAATTCGA-1,TTGTTCAGTGTGCTAC-1,TTGTTGTGTGTCAAGA-1,TTGTTTCACATCCAGG-1,TTGTTTCATTAGTCTA-1,TTGTTTCCATACAACT-1,TTGTTTGTATTACACG-1,TTGTTTGTGTAAATTC-1
ENSG00000243485,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000237613,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000186092,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000238009,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000239945,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8099,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.492611,0.496259,0.494127,0.493547,0.497013,0.495098,0.487406,0.491095,0.497117,0.491866
8100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.486323,0.491544,0.489249,0.490813,0.490854,0.490435,0.479145,0.485391,0.492681,0.487194
8101,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.489216,0.493626,0.492013,0.492649,0.494078,0.493263,0.483633,0.488475,0.495655,0.489331
8102,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.494085,0.497719,0.495215,0.494561,0.498651,0.496119,0.489558,0.492707,0.498208,0.493146


In [28]:
embed_all.loc[8102]

AAACAACGAATAGTTC-1    0.000000
AAACAAGTATCTCCCA-1    0.000000
AAACAATCTACTAGCA-1    0.000000
AAACACCAATAACTGC-1    0.000000
AAACAGCTTTCAGAAG-1    0.000000
                        ...   
TTGTTTCACATCCAGG-1    0.496119
TTGTTTCATTAGTCTA-1    0.489558
TTGTTTCCATACAACT-1    0.492707
TTGTTTGTATTACACG-1    0.498208
TTGTTTGTGTAAATTC-1    0.493146
Name: 8102, Length: 11880, dtype: float64

In [40]:
embed_507_672 = pd.concat([embed_507,embed_672],axis=1)
embed_all = pd.concat([embed_507_672,embed_673],axis=1)
embed_all

,AAACAACGAATAGTTC-1,AAACAAGTATCTCCCA-1,AAACAATCTACTAGCA-1,AAACACCAATAACTGC-1,AAACAGCTTTCAGAAG-1,AAACAGGGTCTATATT-1,AAACAGTGTTCCTGGG-1,AAACATTTCCCGGATT-1,AAACCACTACACAGAT-1,AAACCCGAACGAAATC-1,...,TTGTGTATGCCACCAA-1,TTGTGTTTCCCGAAAG-1,TTGTTAGCAAATTCGA-1,TTGTTCAGTGTGCTAC-1,TTGTTGTGTGTCAAGA-1,TTGTTTCACATCCAGG-1,TTGTTTCATTAGTCTA-1,TTGTTTCCATACAACT-1,TTGTTTGTATTACACG-1,TTGTTTGTGTAAATTC-1
ENSG00000243485,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000237613,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000186092,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000238009,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
ENSG00000239945,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8099,0.404028,0.491486,0.477993,0.490435,0.400040,0.439131,0.474601,0.465490,0.440856,0.432719,...,0.431650,0.508095,0.503970,0.479867,0.504232,0.450862,0.497344,0.471361,0.494474,0.440360
8100,0.334474,0.488156,0.461046,0.481222,0.330221,0.392345,0.457620,0.438856,0.397683,0.379239,...,0.377429,0.516228,0.507249,0.461523,0.505566,0.411650,0.497606,0.451269,0.493267,0.390928
8101,0.321003,0.487434,0.457444,0.479092,0.316786,0.383060,0.454067,0.433428,0.389594,0.368984,...,0.366499,0.517675,0.507676,0.457614,0.505722,0.404450,0.497945,0.446825,0.492925,0.381031
8102,0.418018,0.492266,0.481279,0.492203,0.414148,0.448184,0.477951,0.470595,0.448956,0.443375,...,0.442292,0.506761,0.503513,0.483470,0.504028,0.458276,0.497362,0.475161,0.494960,0.449647


In [30]:
# embed_all.to_csv('dataset/datapoints/embed_all.csv',index = False)

# Save DataFrame to CSV with progress bar
chunks = np.array_split(embed_all.index, 100) # split into 100 chunks

for chunck, subset in enumerate(tqdm(chunks)):
    if chunck == 0: # first row
        embed_all.loc[subset].to_csv('dataset/datapoints/embed_all_com3.csv', mode='w', index=True)
    else:
        embed_all.loc[subset].to_csv('dataset/datapoints/embed_all_com3.csv', header=None, mode='a', index=True)
# embed_all.to_csv('dataset/datapoints/embed_all_com3.csv',index = True, chunksize=1000)


100%|█████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [03:00<00:00,  1.80s/it]


In [31]:
import sys
a = sys.getsizeof(df_spat507)/(1024 ** 2)
b = sys.getsizeof(df_151507)/(1024 ** 2)
c = sys.getsizeof(df_spat672)/(1024 ** 2)
d = sys.getsizeof(df_151672)/(1024 ** 2)
e = sys.getsizeof(df_spat673)/(1024 ** 2)
f = sys.getsizeof(df_151673)/(1024 ** 2)
print(a)
print(b)
print(c)
print(d)
print(e)
print(f)

print(a+b+c+d+e+f)
print(sys.getsizeof(embed_all)/(1024 ** 2))

196.70748901367188
1083.6290893554688
215.43512725830078
1029.6395416259766
224.9944610595703
933.4306793212891
3683.8363876342773
3777.903537750244


In [51]:
import sys
a = sys.getsizeof(df_spat507)/(1024 ** 2)
b = sys.getsizeof(df_151507)/(1024 ** 2)
c = sys.getsizeof(df_spat672)/(1024 ** 2)
d = sys.getsizeof(df_151672)/(1024 ** 2)
e = sys.getsizeof(df_spat673)/(1024 ** 2)
f = sys.getsizeof(df_151673)/(1024 ** 2)
print(a)
print(b)
print(c)
print(d)
print(e)
print(f)

print(a+b+c+d+e+f)
print(sys.getsizeof(embed_all)/(1024 ** 2))

552.9777374267578
1083.6290893554688
637.4516143798828
1029.6395416259766
734.5240631103516
933.4306793212891
4971.652725219727
4967.775676727295


In [23]:
# embed_507.to_csv('datapoints/embedded_507.csv', index=False)
pyreadr.write_rds('datapoints/embed_507.rds', embed_507, compress='gzip')

In [24]:
pyreadr.write_rds('datapoints/embed_672.rds', embed_672, compress='gzip')

In [25]:
pyreadr.write_rds('datapoints/embed_673.rds', embed_673, compress='gzip')

In [13]:
# filters gene with 0 value throughout assay matrix
summ = df_151507.sum(axis=1)
filtered_151507 = df_151507.loc[summ>0]
summ = df_151672.sum(axis=1)
filtered_151672 = df_151672.loc[summ>0]
summ = df_151673.sum(axis=1)
filtered_151673 = df_151673.loc[summ>0]
summ = combined_all.sum(axis=1)
filtered_all = combined_all.loc[summ>0]

# this results in a non-uniform rows, should concate first before filters

In [14]:
# selected gene after filtering non-zero across all samples
df = pd.DataFrame(filtered_all.index.tolist())
selected_gene = df[0].tolist()

In [17]:
len(selected_gene)

23465

In [15]:
df_151673.loc[selected_gene]

,AAACAAGTATCTCCCA-1,AAACAATCTACTAGCA-1,AAACACCAATAACTGC-1,AAACAGAGCGACTCCT-1,AAACAGCTTTCAGAAG-1,AAACAGGGTCTATATT-1,AAACAGTGTTCCTGGG-1,AAACATTTCCCGGATT-1,AAACCCGAACGAAATC-1,AAACCGGGTAGGTACC-1,...,TTGTGTATGCCACCAA-1,TTGTGTTTCCCGAAAG-1,TTGTTAGCAAATTCGA-1,TTGTTCAGTGTGCTAC-1,TTGTTGTGTGTCAAGA-1,TTGTTTCACATCCAGG-1,TTGTTTCATTAGTCTA-1,TTGTTTCCATACAACT-1,TTGTTTGTATTACACG-1,TTGTTTGTGTAAATTC-1
rownames,,,,,,,,,,,,,,,,,,,,,
ENSG00000243485,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000238009,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000229905,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000237491,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000177757,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000278817,0.0,0.0,0.0,0.0,0.804885,0.0,0.0,0.481892,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000277196,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.328571,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000278384,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [19]:
df['coord_list'] = pd.Series(dtype=str)
df['coord_count'] = pd.Series(dtype=int)

In [20]:
df_507 = df.copy()
df_673 = df.copy()
df_672 = df.copy()

In [21]:
df

,0,coord_list,coord_count
0,ENSG00000243485,NaN,NaN
1,ENSG00000238009,NaN,NaN
2,ENSG00000229905,NaN,NaN
3,ENSG00000237491,NaN,NaN
4,ENSG00000177757,NaN,NaN
...,...,...,...
23460,ENSG00000278817,NaN,NaN
23461,ENSG00000277196,NaN,NaN
23462,ENSG00000278384,NaN,NaN
23463,ENSG00000271254,NaN,NaN


<h1>Preprocess

In [22]:
# assay into multiple instances

# read each spots coordinates
coord507 = pyreadr.read_r('spatialLIBD/r_lib/coord_151507.rds')
coord672 = pyreadr.read_r('spatialLIBD/r_lib/coord_151672.rds')
coord673 = pyreadr.read_r('spatialLIBD/r_lib/coord_151673.rds')

# coordinates in respect to gene count values

# output should be matrix of gene distribution for each gene

In [23]:
coord507 = coord507[None]
coord672 = coord672[None]
coord673 = coord673[None]

In [24]:
coord507

,pxl_col_in_fullres,pxl_row_in_fullres
AAACAACGAATAGTTC-1,3276.0,2514.0
AAACAAGTATCTCCCA-1,9178.0,8520.0
AAACAATCTACTAGCA-1,5133.0,2878.0
AAACACCAATAACTGC-1,3462.0,9581.0
AAACAGCTTTCAGAAG-1,2779.0,7663.0
...,...,...
TTGTTGTGTGTCAAGA-1,7464.0,6239.0
TTGTTTCACATCCAGG-1,5045.0,9466.0
TTGTTTCATTAGTCTA-1,4218.0,9703.0
TTGTTTCCATACAACT-1,4017.0,7906.0


In [530]:
coord672

,pxl_col_in_fullres,pxl_row_in_fullres
AAACAAGTATCTCCCA-1,10215.0,8703.0
AAACACCAATAACTGC-1,4499.0,9767.0
AAACAGAGCGACTCCT-1,9675.0,4390.0
AAACAGCTTTCAGAAG-1,3816.0,7849.0
AAACAGGGTCTATATT-1,4090.0,8329.0
...,...,...
TTGTTGTGTGTCAAGA-1,8500.0,6423.0
TTGTTTCACATCCAGG-1,6083.0,9651.0
TTGTTTCATTAGTCTA-1,5256.0,9889.0
TTGTTTCCATACAACT-1,5054.0,8092.0


In [431]:
coord673

,pxl_col_in_fullres,pxl_row_in_fullres
AAACAAGTATCTCCCA-1,9791.0,8468.0
AAACAATCTACTAGCA-1,5769.0,2807.0
AAACACCAATAACTGC-1,4068.0,9505.0
AAACAGAGCGACTCCT-1,9271.0,4151.0
AAACAGCTTTCAGAAG-1,3393.0,7583.0
...,...,...
TTGTTTCACATCCAGG-1,5653.0,9396.0
TTGTTTCATTAGTCTA-1,4825.0,9630.0
TTGTTTCCATACAACT-1,4631.0,7831.0
TTGTTTGTATTACACG-1,5571.0,11193.0


<h5> get coordinate list from assay

In [541]:
# get selected gene from individual sample
select_507 = df_151507.loc[selected_gene]
select_507

,AAACAACGAATAGTTC-1,AAACAAGTATCTCCCA-1,AAACAATCTACTAGCA-1,AAACACCAATAACTGC-1,AAACAGCTTTCAGAAG-1,AAACAGGGTCTATATT-1,AAACAGTGTTCCTGGG-1,AAACATTTCCCGGATT-1,AAACCACTACACAGAT-1,AAACCCGAACGAAATC-1,...,TTGTGTATGCCACCAA-1,TTGTGTTTCCCGAAAG-1,TTGTTAGCAAATTCGA-1,TTGTTCAGTGTGCTAC-1,TTGTTCTAGATACGCT-1,TTGTTGTGTGTCAAGA-1,TTGTTTCACATCCAGG-1,TTGTTTCATTAGTCTA-1,TTGTTTCCATACAACT-1,TTGTTTGTGTAAATTC-1
rownames,,,,,,,,,,,,,,,,,,,,,
ENSG00000243485,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000238009,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000229905,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000237491,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000177757,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ENSG00000278817,0.0,0.841739,0.0,0.0,0.0,0.0,1.382027,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000277196,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.593077,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
ENSG00000278384,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [542]:
# fiters rows from assays, get coordinate list and coordinate count
for i in tqdm(range(len(filtered_all))):   
    a = select_507.iloc[i]
    df_507.loc[i, 'coord_list'] = str(a[a>0].index.tolist())
    df_507.loc[i, 'coord_count'] = len(a[a>0].index.tolist())

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 23465/23465 [00:39<00:00, 597.50it/s]


In [543]:
# Number of gene that have non-zero values, from the initial 33538. should be used as the final shape
df

,0,coord_list,coord_count
0,ENSG00000243485,NaN,NaN
1,ENSG00000238009,NaN,NaN
2,ENSG00000229905,NaN,NaN
3,ENSG00000237491,NaN,NaN
4,ENSG00000177757,NaN,NaN
...,...,...,...
23460,ENSG00000278817,NaN,NaN
23461,ENSG00000277196,NaN,NaN
23462,ENSG00000278384,NaN,NaN
23463,ENSG00000271254,NaN,NaN


In [544]:
df_672

,0,coord_list,coord_count
0,ENSG00000243485,[],0.0
1,ENSG00000238009,"['ACCCTATAGGACTGAG-1', 'ATCGCCAGTCAACATT-1', '...",13.0
2,ENSG00000229905,['GCTAAACCTGAGGTGA-1'],1.0
3,ENSG00000237491,"['AACCAAGACTTCTCTG-1', 'AACCCTACTGTCAATA-1', '...",84.0
4,ENSG00000177757,[],0.0
...,...,...,...
23460,ENSG00000278817,"['AAACCGGAAATGTTAA-1', 'AAACGACAGTCTTGCC-1', '...",620.0
23461,ENSG00000277196,"['AAATTGGTGAGAAGCA-1', 'AAATTTGCGGGTGTGG-1', '...",65.0
23462,ENSG00000278384,"['CCCAACATACGTCGCG-1', 'TATTAACACCAAAGCA-1']",2.0
23463,ENSG00000271254,"['AATTGAACGCTCTGGT-1', 'AGACCATGGGATACAA-1', '...",7.0


In [545]:
df_507

,0,coord_list,coord_count
0,ENSG00000243485,['TTGCCAAGCAGAACCC-1'],1.0
1,ENSG00000238009,"['CGCAATTAGGGTAATA-1', 'CTGCAAGCACGTTCCG-1', '...",6.0
2,ENSG00000229905,[],0.0
3,ENSG00000237491,"['AAAGACCCAAGTCGCG-1', 'AAAGTAGCATTGCTCA-1', '...",57.0
4,ENSG00000177757,['GGAGGGTCAAGTAAGA-1'],1.0
...,...,...,...
23460,ENSG00000278817,"['AAACAAGTATCTCCCA-1', 'AAACAGTGTTCCTGGG-1', '...",530.0
23461,ENSG00000277196,"['AAACCACTACACAGAT-1', 'AACTGGGTTCGAGCCG-1', '...",44.0
23462,ENSG00000278384,"['AGTCACTCCGCCTCAT-1', 'ATACGGTGAAGATGCA-1']",2.0
23463,ENSG00000271254,"['AGAAGGTTGCCGAATT-1', 'AGACTGTTACCGGGTC-1', '...",6.0


<h3>Filter genes with minimum points of coordinates (currently 250)

In [546]:
# filter genes with minimum points of coordinates
df_507_min = df_507.loc[:][df_507.loc[:]['coord_count']>250]

In [547]:
df_672_min

,0,coord_list,coord_count
11,ENSG00000188976,"['AAACAGAGCGACTCCT-1', 'AAACAGGGTCTATATT-1', '...",551.0
16,ENSG00000188290,"['AAACAAGTATCTCCCA-1', 'AAACACCAATAACTGC-1', '...",781.0
17,ENSG00000187608,"['AAACAAGTATCTCCCA-1', 'AAACCGGGTAGGTACC-1', '...",1058.0
18,ENSG00000188157,"['AAACAAGTATCTCCCA-1', 'AAACCCGAACGAAATC-1', '...",647.0
26,ENSG00000078808,"['AAACAGAGCGACTCCT-1', 'AAACCGTTCGTCCAGG-1', '...",953.0
...,...,...,...
23449,ENSG00000198886,"['AAACAAGTATCTCCCA-1', 'AAACACCAATAACTGC-1', '...",4015.0
23450,ENSG00000198786,"['AAACAAGTATCTCCCA-1', 'AAACACCAATAACTGC-1', '...",3877.0
23451,ENSG00000198695,"['AAACCGGAAATGTTAA-1', 'AAACCGGGTAGGTACC-1', '...",1690.0
23452,ENSG00000198727,"['AAACAAGTATCTCCCA-1', 'AAACACCAATAACTGC-1', '...",4015.0


In [548]:
df_507_min

,0,coord_list,coord_count
11,ENSG00000188976,"['AAACCTAAGCAGCCGG-1', 'AAACGAAGAACATACC-1', '...",435.0
16,ENSG00000188290,"['AAACATTTCCCGGATT-1', 'AAACGGGCGTACGGGT-1', '...",709.0
17,ENSG00000187608,"['AAACAACGAATAGTTC-1', 'AAACAAGTATCTCCCA-1', '...",1064.0
18,ENSG00000188157,"['AAACATTTCCCGGATT-1', 'AAACGGGCGTACGGGT-1', '...",534.0
26,ENSG00000078808,"['AAACAGGGTCTATATT-1', 'AAACCACTACACAGAT-1', '...",756.0
...,...,...,...
23449,ENSG00000198886,"['AAACAACGAATAGTTC-1', 'AAACAAGTATCTCCCA-1', '...",4226.0
23450,ENSG00000198786,"['AAACAAGTATCTCCCA-1', 'AAACAATCTACTAGCA-1', '...",4149.0
23451,ENSG00000198695,"['AAACACCAATAACTGC-1', 'AAACAGGGTCTATATT-1', '...",1242.0
23452,ENSG00000198727,"['AAACAACGAATAGTTC-1', 'AAACAAGTATCTCCCA-1', '...",4224.0


In [549]:
df_507_res = df_507_min.reset_index()
df_507_res

,index,0,coord_list,coord_count
0,11,ENSG00000188976,"['AAACCTAAGCAGCCGG-1', 'AAACGAAGAACATACC-1', '...",435.0
1,16,ENSG00000188290,"['AAACATTTCCCGGATT-1', 'AAACGGGCGTACGGGT-1', '...",709.0
2,17,ENSG00000187608,"['AAACAACGAATAGTTC-1', 'AAACAAGTATCTCCCA-1', '...",1064.0
3,18,ENSG00000188157,"['AAACATTTCCCGGATT-1', 'AAACGGGCGTACGGGT-1', '...",534.0
4,26,ENSG00000078808,"['AAACAGGGTCTATATT-1', 'AAACCACTACACAGAT-1', '...",756.0
...,...,...,...,...
6096,23449,ENSG00000198886,"['AAACAACGAATAGTTC-1', 'AAACAAGTATCTCCCA-1', '...",4226.0
6097,23450,ENSG00000198786,"['AAACAAGTATCTCCCA-1', 'AAACAATCTACTAGCA-1', '...",4149.0
6098,23451,ENSG00000198695,"['AAACACCAATAACTGC-1', 'AAACAGGGTCTATATT-1', '...",1242.0
6099,23452,ENSG00000198727,"['AAACAACGAATAGTTC-1', 'AAACAAGTATCTCCCA-1', '...",4224.0


In [550]:
max_row_indices = df_507_res['coord_count'].idxmax()
max_row_indices

6089

In [451]:
df_res.loc[max_row_indices]

index                                                      23440
0                                                ENSG00000198888
coord_list     ['AAACAAGTATCTCCCA-1', 'AAACAATCTACTAGCA-1', '...
coord_count                                               3639.0
Name: 8090, dtype: object

In [525]:
df_672_res.loc[0,0]

'ENSG00000188976'

<h5> from coordinate list into 2D matrix

In [551]:
# for i in range(len(df_res)):

directory = 'datapoints/151507/'

for i in range(len(df_507_res)):

    # get coord_list from dataframe as string back to list
    my_list = ast.literal_eval(df_507_res.loc[i,'coord_list'])

    # table for converting coordinate list into actual pixel coordinates
    ext_coord = coord507.loc[my_list]

    # points = coord673.values.tolist()
    # sites = coord673.index.tolist()
    points = ext_coord.values.tolist()
    sites = ext_coord.index.tolist()
    
    x, y = zip(*points)
    
    # Convert into 2D matrix
    x_resolution = 200
    y_resolution = 250

    x_min = min(x)
    x_max = max(x)
    x_ptp = x_max - x_min

    y_min = min(y)
    y_max = max(y)
    y_ptp = y_max - y_min

    img = np.zeros((y_resolution+20, x_resolution+20), dtype=bool)

    for k, l in points:
        x_pxl = (k - x_min) / x_ptp * x_resolution
        y_pxl = (l - y_min) / y_ptp * y_resolution
        img[int(y_pxl+10), int(x_pxl+10)] = 1
    
    np.save(directory+df_507_res.loc[i,0]+'.npy',img)

In [465]:
# for i in range(len(points)):
#     for j in range(len(points[i])):
#         points[i][j] = points[i][j] * 0.02
# points[0:4]

In [466]:
# x, y = zip(*points)

In [467]:
# print(points[0],x[0],y[0])

[9791.0, 8468.0] 9791.0 8468.0


In [552]:
# for i, filename in enumerate(os.listdir('datapoints/151673')):
#     print(filename)
directory = 'datapoints/151507'
len(os.listdir(directory))

6101

In [553]:
filenames = os.listdir(directory)
filenames[:3]

['ENSG00000137168.npy', 'ENSG00000139168.npy', 'ENSG00000072518.npy']

In [ ]:
plt.figure(figsize=(8,10))

fig, ax = plt.subplots(3,1,figsize=(12,36))

for i, filename in enumerate(filenames[:3]) :
    if filename.endswith('.npy') and os.path.isfile(os.path.join(directory, filename)):
        new_test = np.load(directory+'/'+filename)   
        ax[i].imshow(new_test, cmap='viridis',origin='lower')

plt.show()

In [ ]:
# 2D matrix are formed, but seems to be too sparse to be learned need to rescale
plt.figure(figsize=(8,10))

fig, ax = plt.subplots(figsize=(8,10))
ax.imshow(img, cmap='viridis',origin='lower')
plt.savefig('plot.png')
plt.show()

In [ ]:
# 2D matrix are formed, but seems to be too sparse to be learned need to rescale
plt.figure(figsize=(8,10))

fig, ax = plt.subplots(figsize=(8,10))
ax.imshow(img, cmap='viridis',origin='lower')
# plt.savefig('plot.png')
plt.show()

In [ ]:
plt.figure(figsize=(8,10))
plt.scatter(x,y,s=5,c='b')
plt.show()

In [202]:
len(sites)

3639

<h1>Feed to NN

In [ ]:
# NN take matrix of gene distribution

# make into tensor

# run NN model

# output should be decoder part of hidden features

In [ ]:
os.environ["CUDA_VISIBLE_DEVICES"]="1"
train_root_path = '/home/Spatial_transcriptomic/VERIFY/train' 
test_root_path = '/home/Spatial_transcriptomic/VERIFY/test'
batch_size = 300
epochs = 200
learning_rate = 1e-3
im_size = size_10
# torch.manual_seed(42)

In [ ]:
instance_X_train, instance_X_test = train_test_split(instance_imgs_10, test_size=0.20, random_state=42,shuffle=True) 


train_dataset = TumorDatasetTrain(instance=instance_X_train, root_dir=train_root_path,
                                transform=transforms.Compose([ToTensor()]))
test_dataset = TumorDatasetTest(instance=instance_X_test, root_dir=test_root_path, 
                                transform=transforms.Compose([ToTensor()]))
all_dataset = TumorDatasetAll(instance=instance_imgs_10, root_dir=train_root_path,
                                transform=transforms.Compose([ToTensor()]))

dataloader_all = DataLoader(all_dataset, batch_size=batch_size, shuffle=True, num_workers=1)
dataloader_train = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=1)
dataloader_test = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=1)

running_loss = []
running_vloss = []

In [ ]:
model_path1 = "/home/Spatial_transcriptomic/VERIFY/lab06_AE_weight_decay.pth"
save = False
pret = False

In [ ]:
#  use gpu if available
start_time1 = time.time()

device = torch.device("cuda")

# create a model from `AE` autoencoder class
# load it to the specified device, either gpu or cpu
# model = ConvAutoencoder().to(device)
model = AE(input_shape=im_size*im_size*33).to(device)
model.double()

# create an optimizer object
# Adam optimizer with learning rate 1e-3
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# mean-squared error loss
criterion = nn.MSELoss()
running_loss = []
running_vloss = []

#     Auto Encoder Training
print('Training AE')
if (not os.path.exists(model_path1) or not pret):
    for epoch in range(epochs):
        start_time = time.time()
        loss = 0
        vloss = 0
        model.train()
        for batch_ndx, batch_features in enumerate(dataloader_train):
            # reshape mini-batch data to [N, 784] matrix
            # load it to the active device
    #             batch_features = Variable(batch_features['image']).to(device)
    #             batch_features = batch_features.type(torch.cuda.FloatTensor)
            images = Variable(batch_features['image']).view(-1,im_size*im_size*33).to(device)
    #             images = images.type(torch.cuda.FloatTensor)
            # reset the gradients back to zero
            # PyTorch accumulates gradients on subsequent backward passes
            optimizer.zero_grad()

            # compute reconstructions
            outputs, coded = model(images)
#             print('size', outputs.size(), coded.size())
    #             encoder_only_nosig = model.encoder(images)
    #             encoder_only = torch.sigmoid(model.encoder(images))
            # compute training reconstruction loss
            train_loss = criterion(outputs, images)

            # compute accumulated gradients
            train_loss.backward()

            # perform parameter update based on current gradients
            optimizer.step()
            # add the mini-batch training loss to epoch loss
            loss += train_loss.item()

        # compute the epoch training loss
        loss = loss / len(dataloader_train)
        running_loss.append(loss)
        # display the epoch training loss
        #if(epoch % 5 == 0):
    #       ------------------------------------------------------------------------  
        model.eval()
        with torch.no_grad():
            for batch_ndx, batch_features in enumerate(dataloader_test):
                # reshape mini-batch data to [N, 784] matrix
                # load it to the active device
    #             batch_features = Variable(batch_features['image']).to(device)
    #             batch_features = batch_features.type(torch.cuda.FloatTensor)
                images = Variable(batch_features['image']).view(-1,im_size*im_size*33).to(device)
    #                 images = images.type(torch.cuda.FloatTensor)
                # reset the gradients back to zero
                # PyTorch accumulates gradients on subsequent backward passes
                optimizer.zero_grad()

                # compute reconstructions
                outputs, coded = model(images)
    #             encoder_only_nosig = model.encoder(images)
    #             encoder_only = torch.sigmoid(model.encoder(images))
                # compute training reconstruction loss
                valid_loss = criterion(outputs, images)

                # perform parameter update based on current gradients
    #                 optimizer.step()
                # add the mini-batch training loss to epoch loss
                vloss += valid_loss.item()

            # compute the epoch training loss
            vloss = vloss / len(dataloader_test)
            running_vloss.append(vloss)

    #     -----------------------------------------------------------------------------
        current_time = time.time()
        elapsed_time = current_time - start_time
        print("epoch : {}/{}, recon loss = {:.8f}, valid loss = {:.8f}, time = {:.2f}s".format(epoch + 1, epochs, loss, vloss, elapsed_time))


        if (epoch > 3) and (abs(running_vloss[epoch] - running_vloss[epoch-1]) <= 0.0001) and (abs(running_vloss[epoch - 1] - running_vloss[epoch - 2]) <= 0.0001):
            print('satisfied early stop')
            break
    if save:
        torch.save(model.state_dict(), model_path1)

In [123]:
coord_list = [[1.2, 3.4], [2.3, 4.5], [3.4, 5.6], [4.5, 6.7]]

# convert the list to a PyTorch tensor
coords = torch.tensor(coord_list)

# reshape the tensor to the required input shape (batch size, number of channels, height, width)
coords = coords.view(1, 1, -1, 2)

In [124]:
coords

tensor([[[[1.2000, 3.4000],
          [2.3000, 4.5000],
          [3.4000, 5.6000],
          [4.5000, 6.7000]]]])

<h1>Run BEER

In [ ]:
# get decoder hidden feature and concatenate with assays

# final matrix includes ~100 extra features of spatial embeddings treated as spots

# BEER should results in a Umap plot, Correlation plot, and no-Batch effect umap